<a href="https://colab.research.google.com/github/hchaparov/Dynamic_Pricing_MARL/blob/main/2A_Pong_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install swig
!pip install "pettingzoo"
!pip install "stable-baselines3"
!pip install "supersuit"
!pip install pymunk
!pip install pyvirtualdisplay imageio[ffmpeg]
!apt-get install -y xvfb
!pip install tensorboard
!pip install pillow
!pip install multi_agent_ale_py
!pip install autorom
!pip install moviepy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
ERROR: Operation cancelled by user
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.10).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.10/dist-packages/AutoROM/roms
	/usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/adventure.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/air_raid.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/air_raid.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/alien.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/alien.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/amidar.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/amidar.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/assault.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_

In [ ]:
from __future__ import annotations

from pyvirtualdisplay import Display
import imageio
import numpy as np
import glob
import os
import time
import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.ppo import CnnPolicy
from pettingzoo.atari import pong_v3
from PIL import Image
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback, BaseCallback
from stable_baselines3.common.logger import configure
from moviepy.editor import ImageSequenceClip
from stable_baselines3.common.vec_env import VecVideoRecorder
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

log_dir = "/content/drive/MyDrive/2A_pong_ppo/"
os.makedirs(log_dir, exist_ok=True)

temp_eval_log_dir = "/content/drive/My Drive/2A_pong_ppo/tmp"
os.makedirs(temp_eval_log_dir, exist_ok=True)

n_training_envs = 4

  np.bool8: (False, True),

  from scipy.ndimage.filters import sobel



Mounted at /content/drive


In [ ]:
def train_pong_supersuit(env_fn, steps: int = 200_000, seed: int | None = 0, **env_kwargs):
    # Train a single model to play as each agent in a parallel environment
    env = env_fn.parallel_env(**env_kwargs)
    env = ss.color_reduction_v0(env, mode='full')
    env = ss.resize_v1(env, x_size=84, y_size=84)
    env = ss.frame_stack_v1(env, 4)

    env.reset(seed = seed)

    eval_env = ss.pettingzoo_env_to_vec_env_v1(env)
    eval_env = ss.concat_vec_envs_v1(eval_env, n_training_envs, num_cpus=2, base_class="stable_baselines3")

    env = ss.pettingzoo_env_to_vec_env_v1(env)
    env = ss.concat_vec_envs_v1(env, n_training_envs, num_cpus=2, base_class="stable_baselines3")


    print(f"Starting training")

    # Create callback that evaluates agent for 10 episodes every 20000 training environment steps.
    eval_callback = EvalCallback(eval_env, best_model_save_path=temp_eval_log_dir,
                             log_path=temp_eval_log_dir, eval_freq=max(20000 // n_training_envs, 1),
                             n_eval_episodes=10, deterministic=True,
                             render=False)

    if os.path.exists("/content/drive/MyDrive/2A_pong_ppo/ppo_model.zip"):
      model = PPO.load("/content/drive/MyDrive/2A_pong_ppo/ppo_model_2mil", env=env)
    else:
      model = PPO(
            CnnPolicy,
            env,
            learning_rate=3e-4,  # Lower learning rate for stable training
            n_steps=2048,        # Increase number of steps per update
            batch_size=64,       # Batch size for each update
            n_epochs=10,         # Number of epochs to optimize the surrogate loss
            gamma=0.99,          # Discount factor
            gae_lambda=0.95,     # Factor for trade-off of bias vs variance for Generalized Advantage Estimator
            clip_range=0.2,      # Clipping parameter
            ent_coef=0.1,       # Coefficient for entropy
            vf_coef=0.5,         # Coefficient for value function loss
            max_grad_norm=0.5,   # Maximum norm for gradient clipping
            verbose=1,
        )

    model.learn(total_timesteps=steps, callback=eval_callback)

    model.save("/content/drive/MyDrive/2A_pong_ppo/ppo_model_2mil")

    print("Model has been saved.")
    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.")
    env.close()

  and should_run_async(code)



In [ ]:
def resize_frame(frame, macro_block_size=16):
    height, width, _ = frame.shape
    new_height = (height + macro_block_size - 1) // macro_block_size * macro_block_size
    new_width = (width + macro_block_size - 1) // macro_block_size * macro_block_size
    resized_frame = np.array(Image.fromarray(frame).resize((new_width, new_height)))
    return resized_frame

In [ ]:
def eval(env_fn, num_games: int = 100, render_mode: str | None = None, **env_kwargs):
    # Evaluate a trained agent vs a random agent
    env = env_fn.env(render_mode="rgb_array", **env_kwargs)
    env = ss.color_reduction_v0(env, mode='full')
    env = ss.resize_v1(env, x_size=84, y_size=84)
    env = ss.frame_stack_v1(env, 4)

    print(f"\nStarting evaluation")

    try:
        latest_policy = max(glob.glob("/content/drive/MyDrive/2A_pong_ppo/ppo_model_2mil.zip"))
    except ValueError:
        print("Policy not found.")
        exit(0)

    model = PPO.load(latest_policy)

    rewards = {agent: 0 for agent in env.possible_agents}

    frames = []

    # train using the Parallel API but evaluate using the AEC API
    # using he same SB3 model for every agent
    for i in range(num_games):
        env.reset(seed=i)

        for agent in env.agent_iter():
            obs, reward, termination, truncation, info = env.last()

            for a in env.agents:
                rewards[a] += env.rewards[a]
            if termination or truncation:
                break
            else:
                act = model.predict(obs, deterministic=True)[0]

            env.step(act)

            if render_mode == "human":
                frame = env.render()
                resized_frame = resize_frame(frame)
                frames.append(resized_frame)
    env.close()

    avg_reward = sum(rewards.values()) / len(rewards.values())
    print("Rewards: ", rewards)
    print(f"Avg reward: {avg_reward}")
    if render_mode == "human":
      # Create a video from frames using moviepy
        clip = ImageSequenceClip(frames, fps=10)
        clip.write_videofile('/content/drive/MyDrive/2A_pong_ppo/video_2mil.mp4')

        print("Evaluation saved as /content/drive/MyDrive/2A_pong_ppo/video_2mil.mp4")
    return avg_reward

In [ ]:
def plot():
    # Load existing evaluation results if they exist
    results_file = os.path.join(log_dir, 'evaluations.npz')
    if os.path.exists(results_file):
        old_results = np.load(results_file)
        old_timesteps = old_results['timesteps']
        old_results_raw = old_results['results']
    else:
        old_timesteps = np.array([])
        old_results_raw = np.array([]).reshape(0, 5)

    # Load new evaluation results
    new_results_file = os.path.join(temp_eval_log_dir, 'evaluations.npz')
    new_results = np.load(new_results_file)
    new_timesteps = new_results['timesteps']
    new_results_raw = new_results['results']

    # Append new results to old results
    if old_timesteps.size > 0:
        combined_timesteps = np.concatenate((old_timesteps, new_timesteps + old_timesteps[-1]))
        combined_results_raw = np.concatenate((old_results_raw, new_results_raw))
    else:
        combined_timesteps = new_timesteps
        combined_results_raw = new_results_raw

    # Save combined results
    np.savez(results_file, timesteps=combined_timesteps, results=combined_results_raw)

    # Calculate the mean reward for each evaluation point
    combined_results_mean = combined_results_raw.mean(axis=1)

    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(combined_timesteps, combined_results_mean)
    plt.xlabel('Timesteps')
    plt.ylabel('Mean Reward')
    plt.title('Evaluation Rewards Over Time')
    plt.grid()
    plt.show()

In [ ]:
def show_video(video_path):
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'<video width="480" height="320" controls><source src="{data_url}" type="video/mp4"></video>')


In [ ]:

if __name__ == "__main__":
    display = Display(visible=0, size=(1400, 900))
    display.start()

    env_fn = pong_v3
    env_kwargs = {}

    # Train a model (takes ~3 minutes on GPU)
    train_pong_supersuit(env_fn, steps=2000000, seed = 0, **env_kwargs)


  self.pid = os.fork()



Starting training
Using cpu device
Wrapping the env in a VecTransposeImage.


  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")



------------------------------
| time/              |       |
|    fps             | 338   |
|    iterations      | 1     |
|    time_elapsed    | 48    |
|    total_timesteps | 16384 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 70          |
|    iterations           | 2           |
|    time_elapsed         | 465         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.008626994 |
|    clip_fraction        | 0.00519     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.79       |
|    explained_variance   | -0.00295    |
|    learning_rate        | 0.0003      |
|    loss                 | -0.163      |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00134    |
|    value_loss           | 0.0532      |
-----------------------------------------


  warnings.warn(



Eval num_timesteps=40000, episode_reward=-416.00 +/- 419.00
Episode length: 100000.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 1e+05        |
|    mean_reward          | -416         |
| time/                   |              |
|    total_timesteps      | 40000        |
| train/                  |              |
|    approx_kl            | 0.0054349527 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.79        |
|    explained_variance   | 4.17e-07     |
|    learning_rate        | 0.0003       |
|    loss                 | -0.158       |
|    n_updates            | 20           |
|    policy_gradient_loss | 0.000156     |
|    value_loss           | 0.0453       |
------------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 9     |
|    iterations 

In [ ]:
    plot()

In [ ]:
    # Evaluate 10 games (average reward should be positive but can vary significantly)
    eval(env_fn, num_games=10, render_mode=None, **env_kwargs)
    # Watch 2 games
    eval(env_fn, num_games=2, render_mode="human", **env_kwargs)
    # Display the video after evaluation
    video_path = "/content/drive/MyDrive/2A_pong_ppo/video_2mil.mp4"
    display.stop()

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Convert mp4 to base64
video_path = "/content/drive/MyDrive/2A_pong_ppo/video_2mil.mp4"
video = open(video_path, "rb").read()
video_encoded = b64encode(video).decode()

# Display video
HTML(f"""
<video width="640" height="480" controls>
      <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
</video>
""")
